# Laboratorio 2025
### Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1


In [104]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
from pydistsim.restrictions.topological import Connectivity
from pydistsim.restrictions.knowledge import InitialDistinctValues

from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger, logger
from pydistsim.network.behavior import NetworkBehaviorModel

from pydistsim.gui import drawing as draw
%matplotlib inline
from matplotlib import pyplot as plt
from pydistsim.network import Node

import random
import math

set_log_level(LogLevels.INFO)
enable_logger()

from utils import *

## Parte 1: 

### 1. Implementación básica del problema
- Simule un conjunto de n generales (nodos) que deben decidir si atacar o retirarse.
- Permita que hasta f generales sean bizantinos, es decir, que puedan enviar mensajes contradictorios.
- Los generales leales deberán acordar una decisión común, cumpliendo las condiciones de consistencia y validez.


In [110]:
class ByzantineGenerals(NodeAlgorithm):
    def __init__(self, simulation, *args, f=None, **kwargs):
        super().__init__(simulation, *args, **kwargs)
        self.f = f

    default_params = {
        "Observation" : "Observation",
        "Decision" : "Decision",
    }

    class Status(StatusValues):
        COMMANDER = "COMMANDER"
        LIUTENANT = "LIUTENANT"
        AWAITING_ORDERS = "AWAITING_ORDERS"
        TRAITOR = "TRAITOR"
        ATTACK = "ATTACK"
        RETREAT = "RETREAT"

    S_init = (Status.COMMANDER, Status.LIUTENANT, Status.TRAITOR)
    S_term = (Status.ATTACK, Status.RETREAT)

    algorithm_restrictions = (
        # Restriccion no especificada: grafo completo
        BidirectionalLinks,
        Connectivity, 
        TotalReliability, 
        InitialDistinctValues
    )
        
    def initializer(self):
        self.apply_restrictions()
        n = len(self.network.nodes())
        traidores = random.sample(list(self.network.nodes()), self.f)
        siege = Siege(n)
        commander_general = self.network.nodes_sorted()[0]
        for node in self.network.nodes():
            node.memory["siege"] = siege
            node.memory["commander"] = commander_general #FIXME esto no va a andar así: el teniente precisa su etiqueta a commander, no el objeto Nodo
            node.memory["decisions"] = {id : None for id in map(lambda n : n._internal_id, self.network.nodes())} # Puede que eventualmente solo el comandante tenga esto?
            node.memory["decision_threshold"] = define_general_threshold(node)
            node.memory["traitor"] = node in traidores
            node.status = self.Status.LIUTENANT
            node.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=node))
        commander_general.status = self.Status.COMMANDER


    def error(self, method : str, message: Message):
        msj = 'Unexpected message in ' + method + message.header + " from " + str(message.source) + " , content: " + str(message.data)
        raise Exception(msj)


    def observe(self, node: NodeAccess):
        d = random.random()
        should_retreat = d < node.memory["decision_threshold"]
        logger.info(
            "[{}] Observes they should {}" , 
            f"General {node.memory['unique_value']}", 
            "retreat" if should_retreat else "attack"
        )
        if should_retreat:
            return GeneralDecision.RETREAT
        return GeneralDecision.ATTACK


    def decide(self, node: NodeAccess):
        if len([1 for (id, decision) in node.memory["decisions"].items() if decision == GeneralDecision.ATTACK]) >= int((len(node.neighbors()) + 1)/2):
            return GeneralDecision.ATTACK
        return GeneralDecision.RETREAT
 

    @Status.LIUTENANT
    def spontaneously(self, node: NodeAccess, _: Message):
        node.memory["decisions"][node.memory["unique_value"]] = self.observe(node)
        if node.memory["traitor"]:
            TraitorActions.init(node, self)
        else:
            self.send(
                node, 
                data=(node.memory["decisions"][node.memory["unique_value"]], node.memory["unique_value"]),
                destination=node.memory["commander"],
                header=self.default_params["Observation"]
            )
        node.status = self.Status.AWAITING_ORDERS


    @Status.COMMANDER
    def spontaneously(self, node: NodeAccess, _: Message):
        logger.info("[{}] Is the commander general", f"General {node.memory['unique_value']}")
        node.memory["decisions"][node.memory["unique_value"]] = self.observe(node)
    

    @Status.LIUTENANT
    def receiving(self, node: NodeAccess, message: Message):
        return self.error("LIUTENANT::receiving")


    @Status.COMMANDER
    def receiving(self, node: NodeAccess, message: Message):
        general_decision, general_id = message.data
        node.memory["decisions"][general_id] = general_decision
        if all(decision is not None for (id,decision) in node.memory["decisions"].items() if id != node.memory["unique_value"]):
            commander_decision = self.decide(node)
            if node.memory["traitor"]:
                TraitorActions.commander_decision(node, self)
            else:
                siege : Siege = node.memory["siege"]
                self.send(
                    node, 
                    data=commander_decision,
                    destination=list(node.neighbors()),
                    header=self.default_params["Decision"]
                )
                if commander_decision == GeneralDecision.ATTACK:
                    siege.attack(node)
                    node.status = self.Status.ATTACK
                else:
                    siege.retreat(node)
                    node.status = self.Status.RETREAT

    @Status.AWAITING_ORDERS
    def receiving(self, node: NodeAccess, message: Message):
        commander_decision = message.data
        siege : Siege = node.memory["siege"]
        if node.memory["traitor"]:
            TraitorActions.respond_order(node, self) 
        if commander_decision == GeneralDecision.RETREAT:
            siege.retreat(node)
            node.status = self.Status.RETREAT
        elif commander_decision == GeneralDecision.ATTACK:
            siege.attack(node)
            node.status = self.Status.ATTACK
        else:
            self.error("AWAITING_ORDERS::receiving")

    @Status.ATTACK
    def default(self, *args, **kwargs):    
        pass

    @Status.RETREAT
    def default(self, *args, **kwargs):    
        pass


In [109]:
f = 0
n = 4

net_gen = NetworkGenerator(directed=False)
net = net_gen.generate_complete_network(n)
sim = Simulation(net, check_restrictions=True)
sim.algorithms = ((ByzantineGenerals, {"f": f}),)


sim.run()
# fig = draw.draw_current_state(sim)
# fig

2025-11-21 13:57:44.512 | INFO     | pydistsim.simulation:__init__:59 - Simulation 0x2329de621b0 created successfully.
2025-11-21 13:57:44.517 | INFO     | __main__:spontaneously:89 - [General 317] Is the commander general
2025-11-21 13:57:44.519 | INFO     | __main__:observe:56 - [General 317] Observes they should retreat
2025-11-21 13:57:44.521 | INFO     | __main__:observe:56 - [General 318] Observes they should retreat
2025-11-21 13:57:44.522 | INFO     | __main__:observe:56 - [General 319] Observes they should attack
2025-11-21 13:57:44.523 | INFO     | __main__:observe:56 - [General 320] Observes they should retreat
2025-11-21 13:57:44.524 | INFO     | utils:retreat:33 - [General 317] Retreating
2025-11-21 13:57:44.525 | INFO     | utils:retreat:33 - [General 318] Retreating
2025-11-21 13:57:44.526 | INFO     | utils:retreat:33 - [General 319] Retreating
2025-11-21 13:57:44.526 | INFO     | utils:retreat:33 - [General 320] Retreating
2025-11-21 13:57:44.527 | INFO     | utils:att

In [ ]:
sim.run()

# fig = draw.draw_current_state(sim)
# fig


### 2. Protocolos de comunicación
- Implemente el protocolo recursivo propuesto por Lamport, Shostak y Pease (OM(f)).
- Muestre como crece la complejidad en mensajes según la cantidad de fallos f. Evalúe el caso f = 1 y luego f = 2.
- Mida el número total de mensajes intercambiados y el tiempo necesario para llegar al consenso.

### 3. Análisis
- Determine experimentalmente el número mínimo de nodos necesario para alcanzar consenso frente a f fallos bizantinos.
- Verifique la condición teórica n ≥ 3f + 1.

## Otros

### Otros

In [ ]:
sim.reset()
plt.close()